In [1]:
import pickle

from pyboost import controllers as pb
from pyspark import SparkContext


# sc = SparkContext(master="local[3]")
sc = SparkContext(master=master_url, pyFiles=["dist/pyboost-0.1-py2.7.egg"])
sc.setCheckpointDir('checkpoint/')

In [2]:
# X_train, X_test, y_train, y_test = pickle.load(open("../experiments/higgs/data/higgs-data-1p.pickle"))
X_train, X_test, y_train, y_test = pickle.load(open("../../higgs-data-1p.pickle"))

In [3]:
X = sc.parallelize(X_train)
y = sc.parallelize([y + y - 1 for y in  y_train])

In [4]:
num_round = 1000

In [ ]:
adaboost = pb.run_adtree_adaboost(sc, y, X, T=num_round, quiet=False)

=== Iteration 1 ===
Min score: 0.967118633849
Purity (farther from 1.0 is better): (1.3052988716944389, 0.47728705697035173)
Predicts (farther from 0.0 is better): (0.13321601750449216, -0.36981858625859865)
Split node: 0 (left)
Split index and value: 25 1.1076529026 

=== Iteration 2 ===
Min score: 0.98316527691
Purity (farther from 1.0 is better): (0.66859705334231379, 1.3860486629931645)
Predicts (farther from 0.0 is better): (-0.20128685610574179, 0.16322850526801749)
Split node: 1 (left)
Split index and value: 25 0.738965153694 

=== Iteration 3 ===
Min score: 0.988463402842
Purity (farther from 1.0 is better): (0.46245733677688566, 1.2503552521285859)
Predicts (farther from 0.0 is better): (-0.38560048554560233, 0.1117138563197123)
Split node: 2 (right)
Split index and value: 26 0.833097785711 

=== Iteration 4 ===
Min score: 0.985341652558
Purity (farther from 1.0 is better): (1.3293505658224798, 0.73817551560065175)
Predicts (farther from 0.0 is better): (0.14234526330204483, -

In [ ]:
logitboost = pb.run_adtree_logitboost(sc, y, X, T=num_round, quiet=False)

In [ ]:
import pickle

pickle.dump(adaboost, open("adaboost.pkl", 'w'))
pickle.dump(logitboost, open("logitboost.pkl", 'w'))

# Performance

In [6]:
from pyboost.adtree import run_tree
from pyboost.utils import safe_comp

def test_error(nodes, y, X, max_index=None):    
    error = 0
    margin = 0.0
    for tX, ty in zip(X, y):
        ty = ty + ty - 1
        _m = run_tree(0, nodes, tX, max_index=max_index)
        predict = safe_comp(_m)
        if safe_comp(predict, ty):
            error += 1
        margin += _m * ty
    return 1.0 - float(error) / len(y), margin / len(y)


def test_all(nodes):
    train_accuracy, train_margin = test_error(nodes, y_train, X_train)
    test_accuracy, test_margin = test_error(nodes, y_test, X_test)
    print "Train accuracy:", train_accuracy, '\t',
    print "Train margin:", train_margin
    print "Test accuracy:", test_accuracy, '\t',
    print "Test margin:", test_margin


def test_margin(nodes):
    for max_index in range(num_round + 1):
        accuracy, margin = test_error(nodes, y_train, X_train, max_index=max_index)
        print max_index, accuracy, margin

## Performance of AdaBoost

In [7]:
test_all(adaboost)
test_margin(adaboost)

Train accuracy: 0.66237 	Train margin: 0.18171254852
Test accuracy: 0.6554 	Test margin: 0.17577454326
0 0.52977 0.00354920956848
1 0.61094 0.0516427946144
2 0.63479 0.0798423159376
3 0.63479 0.0968911992297
4 0.63678 0.111258865365
5 0.63816 0.124004737682
6 0.65063 0.144300736904
7 0.65887 0.156041353027
8 0.65741 0.167961583287
9 0.65784 0.175234811317
10 0.66237 0.18171254852


# Performance of LogitBoost

In [12]:
test_all(logitboost)
test_margin(logitboost)

Train accuracy: 0.64391 	Train margin: 0.277308308117
Test accuracy: 0.6407 	Test margin: 0.266371187466
0 0.52977 0.00354920956848
1 0.61089 0.0535789076303
2 0.61089 0.0950319156935
3 0.61089 0.131287217591
4 0.61089 0.170891795839
5 0.61089 0.19305184883
6 0.62958 0.199758238275
7 0.63312 0.222920250926
8 0.63885 0.250397959361
9 0.63885 0.268250095384
10 0.64391 0.277308308117
